In [1]:
from source.regency import Regency
from source.mapping import Mapping
from source.DQNAgent import DQNAgent
from IPython.display import clear_output
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
%matplotlib notebook

In [3]:
seasons = 1
Game = Regency(True)
clear_output()
Game.change_regent('Ch', Gold_Bars=200)
Game.change_regent('Ch', Regency_Points=200)
Game.Regents[Game.Regents['Regent']=='Ch']

,Regent,Full Name,Bloodline,Culture,Player,Class,Level,Alignment,Race,Str,...,Insight,Deception,Persuasion,Regency Points,Gold Bars,Regency Bonus,Attitude,Alive,Divine,Arcane
16,Ch,The Chimera,,A,False,Archmage,18,CE,Elf,0,...,5,3,3,200,200,5,Aggressive,True,False,True


In [4]:
Game.random_events()
Game.collect_regency_points()
Game.domain_initiative()
Game.collect_gold_bars()
Game.maintenance_costs(Update=True)

Instructions for updating:
Use tf.cast instead.


In [5]:
Game.Provences['Regent'] =  Game.Provences['Regent'].str.replace('CoL', 'Ch')
for p in list(Game.Provences[Game.Provences['Regent']=='Ch']['Provence']):
    _, _, message = Game.domain_action_forge_ley_line('Ch', 'Lyssan', p)
    print(message)

The Chimera forged ley lines from Lyssan to Barniere
The Chimera forged ley lines from Lyssan to Careine
The Chimera forged ley lines from Lyssan to Hamein
The Chimera failed to make a ley line from Lyssan to Lyssan
The Chimera forged ley lines from Lyssan to Mhowe
The Chimera failed to make a ley line from Lyssan to Ruorkhe
The Chimera forged ley lines from Lyssan to Salviene


In [6]:



Game.set_override('Ch', 70)
Game.take_domain_actions()

70 Ch Careine 0 1 0 0 1 1 1 1 1e 12
Time to Finish the Rounditiative  3
0


In [10]:
df = Game.Seasons[0]['Actions'][1]
print(df[df['Regent'].str.contains('Ch')]['Message'].values)

['The Chimera applies a tax or asset seizure, and gains 4 gold bars.'
 "The Chimera cast 'Stronghold' in Barniere creating 'Drakessquare', a level 1 permanent castle."]


In [8]:
Game.LeyLines

,Regent,Provence,Other
0,Ch,Lyssan,Barniere
1,Ch,Barniere,Lyssan
2,Ch,Lyssan,Careine
3,Ch,Careine,Lyssan
4,Ch,Lyssan,Hamein
5,Ch,Hamein,Lyssan
6,Ch,Hamein,Mhowe
7,Ch,Mhowe,Hamein
8,Ch,Lyssan,Salviene
9,Ch,Salviene,Lyssan


In [9]:
Game.Provences[Game.Provences['Castle']>0]

,Provence,Domain,Region,Regent,Terrain,Loyalty,Taxation,Population,Magic,Castle,Castle Name,Capital,Position,Contested,Waterway,Brigands
33,Barniere,The Chimaeron,The Eastern Marches,Ch,Plains,High,None,1,4,1,Drakessquare,False,"[1798, 1906]",False,True,False
